In [ ]:
import numpy as np
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("/Users/lindseygulden/dev/leg-up/projects/amperon/data")

PROB = "probability_estimates.csv"
LOAD = "load_hist_data.csv"
WEATHER = "weather_data.csv"

In [ ]:
prob_df = pd.read_csv(DATA_DIR / Path(PROB))
load_df = pd.read_csv(DATA_DIR / Path(LOAD))
weather_df = pd.read_csv(DATA_DIR / Path(WEATHER))

### Load

In [ ]:
load_df.dtypes

In [ ]:
load_df.isna().sum()

In [ ]:
load_df.describe()

In [ ]:
load_df["timestamp"] = pd.to_datetime(load_df.Date) + pd.to_timedelta(
    load_df.Hour, unit="h"
)

In [ ]:
load_df.timestamp.min()

In [ ]:
load_df.timestamp.max()

In [ ]:
plt.figure(figsize=(15, 3))
sns.lineplot(data=load_df, x="timestamp", y="Load")
plt.show()

In [ ]:
sns.boxplot(data=load_df, x="Hour", y="Load")
plt.title("Diurnal cycle")

In [ ]:
load_df["month"] = load_df.timestamp.dt.month
sns.boxplot(data=load_df, x="month", y="Load")
plt.title("Seasonality")

In [ ]:
load_df["dayofweek"] = load_df.timestamp.dt.dayofweek
sns.boxplot(data=load_df, x="dayofweek", y="Load")
plt.title("Day of week variability")

Distribution of peak hour

In [ ]:
idx = (
    load_df.groupby(pd.Grouper(key="timestamp", freq="1D"))["Load"].transform(max)
    == load_df["Load"]
)
peak_load_df = load_df[idx]
peak_load_df

In [ ]:
sns.boxplot(data=peak_load_df, x="month", y="Hour")

In [ ]:
peak_load_df["day"] = peak_load_df.timestamp.dt.day
peak_load_df.groupby(["month", "day"])["Hour"].mean()

In [ ]:
plt.figure(figsize=(15, 5))
sns.heatmap(
    peak_load_df.groupby(["month", "day"])["Hour"]
    .mean()
    .reset_index()
    .pivot(columns="day", index="month", values="Hour"),
    cmap="Spectral",
    vmin=1,
    vmax=24,
    center=12,
)
plt.title("Average Peak Hour")

In [ ]:
peak_load_df.groupby(["month", "day"])["Hour"].std()

In [ ]:
plt.figure(figsize=(15, 5))
sns.heatmap(
    peak_load_df.groupby(["month", "day"])["Hour"]
    .std()
    .reset_index()
    .pivot(columns="day", index="month", values="Hour"),
    cmap="magma",
    vmin=0,
    vmax=8,
)
plt.title("Standard Deviation Peak Hour")

### Weather

In [ ]:
weather_df

In [ ]:
weather_df.dtypes

In [ ]:
weather_df.isna().sum()

In [ ]:
weather_df.duplicated().sum()

In [ ]:
weather_df = weather_df.drop_duplicates()

In [ ]:
weather_df.describe()

In [ ]:
weather_df["timestamp"] = pd.to_datetime(weather_df.Date) + pd.to_timedelta(
    weather_df.Hour, unit="h"
)

In [ ]:
weather_df.timestamp.min()

In [ ]:
weather_df.timestamp.max()

In [ ]:
# weather_df=weather_df[['Date',,'Station ID','Temperature']].groupby(['timestamp','Station ID']).mean().reset_index()
weather_df = (
    weather_df.groupby(["timestamp", "Station ID"])
    .agg({"Date": "first", "Hour": "first", "Temperature": "mean"})
    .reset_index()
)
len(weather_df)

In [ ]:
# weather_df['Hour']=weather_df.timestamp.dt.hour
weather_df["month"] = weather_df.timestamp.dt.month

In [ ]:
plt.figure(figsize=(15, 3))
sns.lineplot(
    data=weather_df,
    x="timestamp",
    y="Temperature",
    hue="Station ID",
    palette="Spectral",
)
plt.show()

In [ ]:
sns.boxplot(data=weather_df, x="Station ID", y="Temperature", palette="Spectral")

In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(
    data=weather_df, x="month", y="Temperature", hue="Station ID", palette="Spectral"
)

In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(
    data=weather_df, x="Hour", y="Temperature", hue="Station ID", palette="Spectral"
)

### Merge Load and Weather to investigate relationships 

In [ ]:
df_load_weather = weather_df.merge(
    load_df.drop(["Date", "Hour", "month"], axis=1), on="timestamp", how="left"
)

In [ ]:
df_load_weather.columns

In [ ]:
sns.scatterplot(
    data=df_load_weather,
    x="Temperature",
    y="Load",
    hue="Station ID",
    marker=".",
    palette="Spectral",
)

In [ ]:
df_load_weather["HDH"] = [x - 65 if x >= 65 else 0 for x in df_load_weather.Temperature]
df_load_weather["CDH"] = [65 - x if x < 65 else 0 for x in df_load_weather.Temperature]

In [ ]:
sns.scatterplot(
    data=df_load_weather[df_load_weather["Station ID"] == 1],
    x="HDH",
    y="Load",
    marker=".",
)

In [ ]:
sns.scatterplot(
    data=df_load_weather[df_load_weather["Station ID"] == 1],
    x="CDH",
    y="Load",
    marker=".",
)

In [ ]:
df_load_weather

Peak daily load and temperature

In [ ]:
df_peak_load = df_load_weather.copy()
idx = (
    df_peak_load.groupby(pd.Grouper(key="timestamp", freq="1D"))["Load"].transform(max)
    == df_peak_load["Load"]
)
df_peak_load = df_peak_load[idx]  # [['timestamp','Station ID','Temperature','Load']]

In [ ]:
sns.scatterplot(
    data=df_peak_load.sort_values(by="Station ID"),
    x="Temperature",
    y="Load",
    hue="Station ID",
    marker=".",
    palette="Spectral",
)
plt.ylabel("Peak Load")

In [ ]:
sns.scatterplot(
    data=df_peak_load.sort_values(by="Station ID"),
    x="CDH",
    y="Load",
    marker=".",
    hue="Station ID",
    palette="Spectral",
)
plt.ylabel("Peak Load")

In [ ]:
sns.scatterplot(
    data=df_peak_load.sort_values(by="Station ID"),
    x="HDH",
    y="Load",
    marker=".",
    hue="Station ID",
    palette="Spectral",
)
plt.ylabel("Peak Load")

### Linear Model Fit by Station

In [ ]:
df_load_weather.columns

In [ ]:
station = 13
df_load_weather_st = df_load_weather[df_load_weather["Station ID"] == station]

In [ ]:
df_load_weather_st["lagged_TMP_24h"] = df_load_weather_st["Temperature"].shift(24)

In [ ]:
df_load_weather_st["lagged_TMP_1h"] = df_load_weather_st["Temperature"].shift(1)

In [ ]:
df_load_weather_st["lagged_load_24h"] = df_load_weather_st["Load"].shift(24)

In [ ]:
# df_load_weather_st['past_min_load']=
tmp = (
    df_load_weather_st.groupby(pd.Grouper(key="timestamp", freq="1D"))[
        "lagged_load_24h"
    ]
    .min()
    .reset_index()
)
tmp.rename(
    columns={"timestamp": "Datetime", "lagged_load_24h": "previous_min_load"},
    inplace=True,
)

In [ ]:
df_load_weather_st["Datetime"] = pd.to_datetime(df_load_weather_st.Date)
df_load_weather_st = df_load_weather_st.merge(tmp, on="Datetime")

In [ ]:
df_load_weather_st.columns

In [ ]:
df_load_weather_st.isna().sum()

In [ ]:
df_load_weather_st["dayofweek"] = df_load_weather_st.timestamp.dt.dayofweek

In [ ]:
from patsy import dmatrices

formula = "Load ~ Temperature:C(month) + Temperature:C(Hour) + I(Temperature**2) + I(Temperature**3) + CDH + HDH + C(dayofweek):C(Hour)"
# formula = 'Load ~ Temperature:C(month) + Temperature:C(Hour) + I(Temperature**2) + I(Temperature**3) + CDH + HDH + lagged_TMP_24h'
# formula = 'Load ~ Temperature:C(month) + Temperature:C(Hour) + I(Temperature**2) + I(Temperature**3) + CDH + HDH + lagged_TMP_1h'
# formula = 'Load ~ Temperature:C(month) + Temperature:C(Hour) + I(Temperature**2) + I(Temperature**3) + CDH + HDH + lagged_load_24h + C(dayofweek):C(Hour)'
# formula = 'Load ~ Temperature:C(month) + Temperature:C(Hour) + I(Temperature**2) + I(Temperature**3) + CDH + HDH + lagged_load_24h + previous_min_load'
y, X = dmatrices(formula, df_load_weather_st, return_type="dataframe")

In [ ]:
X

In [ ]:
model = LinearRegression()
model.fit(X, y)

In [ ]:
model.n_features_in_

In [ ]:
print("Obs per coeff: " + str(len(X) / model.n_features_in_))

In [ ]:
y_pred = model.predict(X)

In [ ]:
model.score(X, y)

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
ax.scatter(y, y_pred, marker=".")
ax.plot([0, 1], [0, 1], transform=ax.transAxes, color="gray", linestyle="dashed")
plt.xlabel("Observed Load")
plt.ylabel("Predicted Load")
plt.title("with Station ID: " + str(station))

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

mae(y, y_pred)

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
ax.scatter(y, y - y_pred, marker=".")
ax.axhline(0, color="gray", linestyle="dashed")
plt.xlabel("Observed Load")
plt.ylabel("Bias: Observed - Predicted Load")
plt.title("with Station ID: " + str(station))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as mape

mape(y, y_pred)

In [ ]:
# df_subset=df_load_weather_st[~df_load_weather_st.isna().any(axis=1)].copy()  <- for nans in lags
df_subset = df_load_weather_st[df_load_weather_st.Load > 0].copy()
df_subset["Load_predicted"] = y_pred

In [ ]:
plt.figure(figsize=(15, 3))
sns.lineplot(data=df_subset, x="timestamp", y="Load_predicted", label="predicted")
sns.scatterplot(
    data=df_subset, x="timestamp", y="Load", marker=".", color="r", label="observed"
)
plt.ylabel("Hourly Load (MW)")
plt.title("with Station ID: " + str(station))

Check maxima, predicted vs observed

In [ ]:
idx = (
    df_subset.groupby(pd.Grouper(key="timestamp", freq="1D"))[
        "Load_predicted"
    ].transform(max)
    == df_subset["Load_predicted"]
)
df_peak_pred = df_subset[idx][
    ["Datetime", "timestamp", "Date", "Hour", "Load_predicted"]
]

In [ ]:
idx = (
    df_subset.groupby(pd.Grouper(key="timestamp", freq="1D"))["Load"].transform(max)
    == df_subset["Load"]
)
df_peak_obs = df_subset[idx][["Datetime", "timestamp", "Date", "Hour", "Load"]]

In [ ]:
plt.figure(figsize=(15, 3))
sns.scatterplot(
    data=df_peak_pred, x="timestamp", y="Load_predicted", marker=".", label="predicted"
)
sns.scatterplot(
    data=df_peak_obs, x="timestamp", y="Load", marker=".", color="r", label="observed"
)
plt.ylabel("Peak load")
plt.title("with Station ID: " + str(station))

In [ ]:
plt.figure(figsize=(15, 5))
sns.scatterplot(
    data=df_peak_pred, x="Datetime", y="Hour", marker=".", label="predicted"
)
sns.scatterplot(
    data=df_peak_obs, x="Datetime", y="Hour", marker=".", color="r", label="observed"
)
plt.ylabel("Peak hour")
plt.title("with Station ID: " + str(station))

In [ ]:
df_peak_obs

In [ ]:
df_peak_pred

In [ ]:
df_peak_pred_pivot = df_peak_pred.copy()
df_peak_pred_pivot["month"] = df_peak_pred_pivot.timestamp.dt.month
df_peak_pred_pivot["day"] = df_peak_pred_pivot.timestamp.dt.day
plt.figure(figsize=(15, 5))
sns.heatmap(
    df_peak_pred_pivot.groupby(["month", "day"])["Hour"]
    .mean()
    .reset_index()
    .pivot(columns="day", index="month", values="Hour"),
    cmap="Spectral",
    vmin=1,
    vmax=24,
    center=12,
)
plt.title("Predicted Average Peak Hour")

In [ ]:
plt.figure(figsize=(15, 5))
sns.heatmap(
    df_peak_pred_pivot.groupby(["month", "day"])["Hour"]
    .std()
    .reset_index()
    .pivot(columns="day", index="month", values="Hour"),
    cmap="magma",
    vmin=0,
    vmax=8,
)
plt.title("Predicted Std Dev Peak Hour")

### Direct Forecasting 2008 for models without lagged load

Forecast 2008 with lagged load terms needs up to 1 day steps

In [ ]:
df_load_weather_st[df_load_weather_st.timestamp > "2008-01-01"].isna().sum()

In [ ]:
df_forecast = df_load_weather_st[(df_load_weather_st.timestamp > "2008-01-01")].copy()

In [ ]:
df_forecast["Load"] = 0  # sentinel needed otherwise X comes back empty from dmatrices
yy, X_forecast = dmatrices(formula, df_forecast, return_type="dataframe")
y_forecast = model.predict(X_forecast)
df_forecast["Load_predicted"] = y_forecast

In [ ]:
idx = (
    df_forecast.groupby(pd.Grouper(key="timestamp", freq="1D"))[
        "Load_predicted"
    ].transform(max)
    == df_forecast["Load_predicted"]
)
df_forecasted_peaks = df_forecast[idx][["Date", "Hour", "timestamp", "Load_predicted"]]

In [ ]:
plt.figure(figsize=(15, 3))
sns.lineplot(
    data=df_forecast, x="timestamp", y="Load_predicted", color="gray", alpha=0.1
)
sns.scatterplot(
    data=df_forecasted_peaks,
    x="timestamp",
    y="Load_predicted",
    color="r",
    marker=".",
    label="peak",
)

In [ ]:
df_forecasted_peaks.head()

In [ ]:
df_forecast[df_forecast.Date == "1/2/2008"][
    ["Date", "Hour", "timestamp", "Load_predicted"]
]

In [ ]:
df_forecasted_peaks_pivot = df_forecasted_peaks.copy()
df_forecasted_peaks_pivot["day"] = df_forecasted_peaks_pivot.timestamp.dt.day
df_forecasted_peaks_pivot["month"] = df_forecasted_peaks_pivot.timestamp.dt.month
plt.figure(figsize=(15, 5))
sns.heatmap(
    df_peak_pred_pivot.groupby(["month", "day"])["Hour"]
    .first()
    .reset_index()
    .pivot(columns="day", index="month", values="Hour"),
    annot=True,
    annot_kws={"fontsize": 8},
    cmap="Spectral",
    vmin=1,
    vmax=24,
    center=12,
)
plt.title("Predicted 2008 Peak Hour with Station ID:" + str(station))

### Predict with all stations and collect peak hours

In [ ]:
formula = "Load ~ Temperature:C(month) + Temperature:C(Hour) + I(Temperature**2) + I(Temperature**3) + CDH + HDH + C(dayofweek):C(Hour)"

In [ ]:
peak_hours = []
for station in df_load_weather["Station ID"].unique():
    df_load_weather_st = df_load_weather[df_load_weather["Station ID"] == station]
    df_load_weather_st["dayofweek"] = df_load_weather_st.timestamp.dt.dayofweek
    y, X = dmatrices(formula, df_load_weather_st, return_type="dataframe")
    model = LinearRegression()
    model.fit(X, y)
    df_forecast = df_load_weather_st[
        (df_load_weather_st.timestamp > "2008-01-01")
    ].copy()
    df_forecast[
        "Load"
    ] = 0  # sentinel needed otherwise X comes back empty from dmatrices
    yy, X_forecast = dmatrices(formula, df_forecast, return_type="dataframe")
    y_forecast = model.predict(X_forecast)
    df_forecast["Load_predicted"] = y_forecast
    idx = (
        df_forecast.groupby(pd.Grouper(key="timestamp", freq="1D"))[
            "Load_predicted"
        ].transform(max)
        == df_forecast["Load_predicted"]
    )
    df_forecasted_peaks = df_forecast[idx][
        ["Date", "Hour", "timestamp", "Load_predicted"]
    ]
    df_forecasted_peaks["station"] = station
    peak_hours.append(df_forecasted_peaks)

In [ ]:
pd.concat(peak_hours).to_csv("peak_forecast_allstations.csv")